# Pose Detection with AlphaPhose

This notebook uses an open source project [MVIG-SJTU/AlphaPose](https://github.com/MVIG-SJTU/AlphaPose) to detect/track multi person poses on a given youtube video.

For other deep-learning Colab notebooks, visit [tugstugi/dl-colab-notebooks](https://github.com/tugstugi/dl-colab-notebooks).

**Beforw start, please make sure you can use GPU.
「執行階段」 -> 「變更執行階段類型」 -> 「硬體加速器」GPU **

## Install AlphaPose

In [ ]:
# 1. Install PyTorch
!pip3 install torch==1.1.0 torchvision==0.3.0
!pip install Pillow==6.2.2 --user
!pip install -U PyYAML

# 2. Get AlphaPose
!git clone https://github.com/MVIG-SJTU/AlphaPose.git
%cd AlphaPose

# 3. install
!export PATH=/usr/local/cuda/bin/:$PATH
!export LD_LIBRARY_PATH=/usr/local/cuda/lib64/:$LD_LIBRARY_PATH
!pip install cython
!sudo apt-get install libyaml-dev
!python setup.py build develop --user

Requirement already up-to-date: PyYAML in /usr/local/lib/python3.6/dist-packages (5.3.1)
Cloning into 'AlphaPose'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 2038 (delta 0), reused 4 (delta 0), pack-reused 2033
Receiving objects: 100% (2038/2038), 108.36 MiB | 35.26 MiB/s, done.
Resolving deltas: 100% (958/958), done.
/content/AlphaPose/AlphaPose
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libyaml-dev is already the newest version (0.1.7-2ubuntu3).
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.
running build
running build_py
creating build
creating build/lib.linux-x86_64-3.6
creating build/lib.linux-x86_64-3.6/alphapose
copying alphapose/__init__.py -> build/lib.linux-x86_64-3.6/alphapose
copying alphapose/opt.py -> build/lib.linux-x86_64-3.6/alphapose
copying alphapose/version.py -> build/lib.linux-x86_64-3.6/alphapo

## Create Folder

In [ ]:
# Model Weights Folder
!mkdir detector/yolo/data
!mkdir detector/tracker/data
# Output Folder
!mkdir exp
# Dataset Folder
!mkdir data
!mkdir data/coco
!mkdir data/coco/annotations

mkdir: cannot create directory ‘detector/yolo/data’: File exists
mkdir: cannot create directory ‘detector/tracker/data’: File exists
mkdir: cannot create directory ‘exp’: File exists
mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘data/coco’: File exists
mkdir: cannot create directory ‘data/coco/annotations’: File exists


## Download pretrained models

In [ ]:
import os
from os.path import exists, join, basename, splitext
git_repo_url = 'https://github.com/MVIG-SJTU/AlphaPose.git'
project_name = splitext(basename(git_repo_url))[0]

def download_from_google_drive(file_id, file_name):
  # download a file from the Google Drive link
  !rm -f ./cookie
  !curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=$file_id" > /dev/null
  confirm_text = !awk '/download/ {print $NF}' ./cookie
  if(len(confirm_text) > 0):
    confirm_text = confirm_text[0]
  !curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=$confirm_text&id=$file_id" -o $file_name
  

  
tracker_model_path = join('detector/tracker/data/JDE-1088x608-uncertainty')
if not exists(tracker_model_path):
  # download the pretrained model
  print("jiji")
  download_from_google_drive('1nlnuYfGNuHWZztQHXwVZSL_FvfE551pA', tracker_model_path)
  
yolo_pretrained_model_path = join('detector/yolo/data/yolov3-spp.weights')
if not exists(yolo_pretrained_model_path):
  # download the YOLO weights
  download_from_google_drive('1D47msNOOiJKvPOXlnpyzdKA3k6E97NTC', yolo_pretrained_model_path)

pretrained_model_path = join('pretrained_models/fast_res50_256x192.pth')
if not exists(pretrained_model_path):
  # download the YOLO weights
  download_from_google_drive('1kQhnMRURFiy7NsdS8EFL-8vtqEXOgECn', pretrained_model_path)

jiji
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   2103      0 --:--:-- --:--:-- --:--:--  2103
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  557M    0  557M    0     0   203M      0 --:--:--  0:00:02 --:--:--  289M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   1758      0 --:--:-- --:--:-- --:--:--  1758
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  240M    0  240M    0     0  78.4M      0 --:--:--  0:00:03 --:--:--  100M
  % Total    % Received % Xferd  Average Speed 

## Download COCO dataset

In [ ]:
# Annotation File of train, val data
annot_file_path = join('data/coco/annotations/person_keypoints_val2017.json')
if not exists(annot_file_path):
  # download the pretrained model
  download_from_google_drive('1dNUbKtmAr8H-ewmv-amUcvCj66eM_zRH', annot_file_path)
annot_file_path = join('data/coco/annotations/person_keypoints_train2017.json')
if not exists(annot_file_path):
  # download the pretrained model
  download_from_google_drive('1dNUbKtmAr8H-ewmv-amUcvCj66eM_zRH', annot_file_path)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0    523      0 --:--:-- --:--:-- --:--:--   522
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 9785k    0 9785k    0     0  9727k      0 --:--:--  0:00:01 --:--:-- 9727k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0    708      0 --:--:-- --:--:-- --:--:--   708
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 9785k    0 9785k    0     0  11.6M      0 --:--:-- --:--:-- --:--:-- 11.6M


In [ ]:
# COCO image
!wget http://images.cocodataset.org/zips/val2017.zip
!unzip val2017.zip
!mv val2017 data/coco
!cp -r data/coco/val2017 data/coco/train2017

--2020-11-15 08:05:32--  http://images.cocodataset.org/zips/val2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.11.172
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.11.172|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 815585330 (778M) [application/zip]
Saving to: ‘val2017.zip’

val2017.zip         100%[===================>] 777.80M  94.9MB/s    in 9.4s    

2020-11-15 08:05:42 (82.7 MB/s) - ‘val2017.zip’ saved [815585330/815585330]



## Training Model

In [ ]:
!./scripts/train.sh ./configs/coco/resnet/256x192_res50_lr1e-3_1x.yaml exp_fastpose

++ CONFIG=./configs/coco/resnet/256x192_res50_lr1e-3_1x.yaml
++ EXPID=exp_fastpose
++ python ./scripts/train.py --exp-id exp_fastpose --cfg ./configs/coco/resnet/256x192_res50_lr1e-3_1x.yaml
******************************
Namespace(board=True, cfg='./configs/coco/resnet/256x192_res50_lr1e-3_1x.yaml', debug=False, detector='yolo', device=device(type='cuda', index=0), dist_backend='nccl', dist_url='tcp://192.168.1.214:23345', exp_id='exp_fastpose', gpus=[0], launcher='none', map=True, nThreads=60, rank=-1, snapshot=2, sync=False, work_dir='./exp/exp_fastpose-256x192_res50_lr1e-3_1x.yaml/', world_size=4)
******************************
{'DATASET': {'TRAIN': {'TYPE': 'Mscoco', 'ROOT': './data/coco/', 'IMG_PREFIX': 'train2017', 'ANN': 'annotations/person_keypoints_train2017.json', 'AUG': {'FLIP': True, 'ROT_FACTOR': 40, 'SCALE_FACTOR': 0.3, 'NUM_JOINTS_HALF_BODY': 8, 'PROB_HALF_BODY': -1}}, 'VAL': {'TYPE': 'Mscoco', 'ROOT': './data/coco/', 'IMG_PREFIX': 'val2017', 'ANN': 'annotations/person_

##Change Training Setting

1. Go to `configs/coco/resnet`
2. add a `.yaml` file
```
DATASET:
  TRAIN:
    TYPE: 'Mscoco'
    ROOT: './data/coco/'
    IMG_PREFIX: 'train2017'
    ANN: 'annotations/person_keypoints_train2017.json'
    AUG:
      FLIP: true
      ROT_FACTOR: 40
      SCALE_FACTOR: 0.3
      NUM_JOINTS_HALF_BODY: 8
      PROB_HALF_BODY: -1
  VAL:
    TYPE: 'Mscoco'
    ROOT: './data/coco/'
    IMG_PREFIX: 'val2017'
    ANN: 'annotations/person_keypoints_val2017.json'
  TEST:
    TYPE: 'Mscoco_det'
    ROOT: './data/coco/'
    IMG_PREFIX: 'val2017'
    DET_FILE: './exp/json/test_det_yolo.json'
    ANN: 'annotations/person_keypoints_val2017.json'
DATA_PRESET:
  TYPE: 'simple'
  SIGMA: 2
  NUM_JOINTS: 17
  IMAGE_SIZE:
  - 256
  - 192
  HEATMAP_SIZE:
  - 64
  - 48
MODEL:
  TYPE: 'FastPose'
  PRETRAINED: ''
  TRY_LOAD: ''
  NUM_DECONV_FILTERS:
  - 256
  - 256
  - 256
  NUM_LAYERS: 50
LOSS:
  TYPE: 'MSELoss'
DETECTOR:
  NAME: 'yolo'
  CONFIG: 'detector/yolo/cfg/yolov3-spp.cfg'
  WEIGHTS: 'detector/yolo/data/yolov3-spp.weights'
  NMS_THRES: 0.6
  CONFIDENCE: 0.05
TRAIN:
  WORLD_SIZE: 4
  BATCH_SIZE: 32
  BEGIN_EPOCH: 0
  END_EPOCH: 200
  OPTIMIZER: 'adam'
  LR: 0.001
  LR_FACTOR: 0.1
  LR_STEP:
  - 90
  - 120
  DPG_MILESTONE: 140
  DPG_STEP:
  - 160
  - 190
``` 